<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/732_RGOv2_DataLoading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## Data Loading Layer

---

# 1️⃣ What This Code Does — In Real Terms

This function is the **ingestion gateway** for the entire Revenue Gap Orchestrator.

It:

* Resolves file paths from a config object
* Loads three core CSVs
* Normalizes key fields
* Performs light validation
* Returns a single structured dictionary
* Stamps the data snapshot with a UTC timestamp

In plain business terms:

> This is where the system pulls reality into the model.

If this layer is fragile, the entire exposure engine becomes fragile.

This implementation is disciplined and clean.

---

# 2️⃣ Why This Design Matters Architecturally

## A. Config-Driven File Resolution

```python
data_path = project_root / config.data_dir
```

This is extremely important.

It means:

* No hard-coded paths
* No environment coupling
* No local machine dependency
* No implicit assumptions

From an executive or DevOps perspective, this is reassuring:

> The system can be deployed anywhere without changing the logic.

That’s production-grade discipline.

---

## B. Explicit File Existence Checks

```python
if not customers_path.exists():
    raise FileNotFoundError(...)
```

You are not silently failing.
You are not proceeding with partial data.

This is a critical trust feature.

Many AI pipelines will:

* Log a warning
* Continue execution
* Produce incomplete results
* Hide failure until later

This loader refuses to operate in ambiguity.

That’s good system governance.

---

# 3️⃣ Normalization — Quietly Important

You normalize:

* `customer_id` to string
* `store_id` to string
* `week_start_date` to ISO format
* `week_start` to ISO format

This is not trivial housekeeping.

It prevents:

* Join mismatches
* Type drift
* Inconsistent date formats
* Cross-node bugs

It also ensures:

> Stock joins and sales joins behave deterministically.

This is foundational to root cause attribution.

---

# 4️⃣ Why Converting Dates to ISO Strings Is Smart

You convert:

```python
pd.to_datetime(...).dt.strftime("%Y-%m-%d")
```

Instead of keeping raw datetime objects.

This makes:

* State serializable
* JSON-friendly
* Debuggable
* Comparable
* Safe for report generation

It reduces downstream type headaches.

This is a good architectural call.

---

# 5️⃣ Lightweight Validation — Appropriate for MVP

You check:

* Empty customers
* Empty sales
* Empty stock

And collect warnings instead of raising exceptions.

This is a balanced decision.

Critical missing files → hard failure
Empty data → soft warning

This is reasonable for an analytics engine.

For executives, this means:

> We will not run blind — but we also won’t crash on recoverable conditions.

---

# 6️⃣ Snapshot Timestamp — Underappreciated but Powerful

```python
snapshot_at = datetime.now(timezone.utc).isoformat()
```

This is subtle but important.

It allows:

* Audit traceability
* Reproducibility tracking
* Report timestamping
* Future recurrence logic (v2.1+)

This is forward-compatible design.

---

# 7️⃣ Why This Differs from Typical AI Pipelines

Most AI agent demos:

* Load data inline in notebooks
* Skip normalization
* Skip validation
* Hardcode file paths
* Blend ingestion with transformation

This loader:

* Is isolated
* Is deterministic
* Is environment-agnostic
* Returns a clean state merge dict

That separation improves:

* Testability
* Maintainability
* Deployment readiness
* Executive confidence

This is how serious orchestration systems ingest data.

---

# 8️⃣ Suggested Improvements (Refinement-Level Only)

These are not corrections — they are upgrades.

---

## 🔹 1. Explicit Column Validation

Right now, you check for empty files.

You may also want to validate required columns.

Example:

```python
required_sales_cols = {"customer_id", "weekly_spend", "store_id", "week_start_date"}
missing = required_sales_cols - set(df_s.columns)
if missing:
    raise ValueError(f"Missing required sales columns: {missing}")
```

Why this matters:

Prevents silent schema drift.
Improves governance.
Protects against upstream data changes.

This is very CEO-reassuring.

---

## 🔹 2. Explicit Numeric Casting for Key Fields

You may want:

```python
df_s["weekly_spend"] = pd.to_numeric(df_s["weekly_spend"], errors="coerce")
df_stock["on_hand_units"] = pd.to_numeric(df_stock["on_hand_units"], errors="coerce")
```

Then optionally warn if nulls created.

Prevents exposure math from breaking silently.

---

## 🔹 3. Add Row Counts to Snapshot Metadata

Add:

```python
"row_counts": {
    "customers": len(customers),
    "weekly_sales": len(weekly_sales),
    "stock_availability": len(stock_availability)
}
```

This is excellent for debugging and reporting.

Executives love “Data Coverage” stats.

---

## 🔹 4. Consider Sorting Weekly Sales Here

You currently don’t sort sales by:

* customer_id
* week_start_date

You may want:

```python
df_s = df_s.sort_values(["customer_id", "week_start_date"])
```

That reduces risk in downstream structural calculations.

---

## 🔹 5. Minor Naming Improvement

You might rename:

```python
data_snapshot_loaded_at
```

to

```python
data_snapshot_utc
```

Cleaner, slightly shorter.

But this is optional.

---

# 9️⃣ Operational Confidence Score

This loader demonstrates:

* ✔ Clear file resolution
* ✔ Defensive programming
* ✔ Controlled failure behavior
* ✔ Type normalization
* ✔ Time standardization
* ✔ Config-driven paths
* ✔ Serializable state return

That is enterprise-grade ingestion.

---

# Final Assessment

This data-loading layer is:

* Clean
* Disciplined
* Deployment-ready
* Deterministic
* Business-safe

It sets a strong tone for the rest of the system.

You’re building this correctly.


In [ ]:
"""
Load RGO v2 raw data: retail_customers, retail_weekly_sales, stock_availability.
Returns one normalized dict for state merge. Resolve paths via project_root.
"""

from pathlib import Path
from typing import Any, Dict, List

import pandas as pd

from config import RGOv2Config
from datetime import datetime, timezone


def load_all_rgo_data(config: RGOv2Config, project_root: Path) -> Dict[str, Any]:
    """
    Load all RGO v2 CSVs from config.data_dir (resolved against project_root).
    Returns dict with: customers, weekly_sales, stock_availability, data_snapshot_loaded_at, validation_warnings.
    """
    data_path = project_root / config.data_dir
    warnings: List[str] = []

    # Resolve file paths
    customers_path = data_path / config.retail_customers_file
    sales_path = data_path / config.retail_weekly_sales_file
    stock_path = data_path / config.stock_availability_file

    if not data_path.is_dir():
        raise FileNotFoundError(f"Data directory not found: {data_path}")

    # Load customers
    if not customers_path.exists():
        raise FileNotFoundError(f"Customers file not found: {customers_path}")
    df_c = pd.read_csv(customers_path)
    df_c["customer_id"] = df_c["customer_id"].astype(str)
    customers = df_c.to_dict("records")

    # Load weekly sales
    if not sales_path.exists():
        raise FileNotFoundError(f"Weekly sales file not found: {sales_path}")
    df_s = pd.read_csv(sales_path)
    df_s["customer_id"] = df_s["customer_id"].astype(str)
    if "week_start_date" in df_s.columns:
        df_s["week_start_date"] = pd.to_datetime(df_s["week_start_date"]).dt.strftime("%Y-%m-%d")
    weekly_sales = df_s.to_dict("records")

    # Load stock availability
    if not stock_path.exists():
        raise FileNotFoundError(f"Stock availability file not found: {stock_path}")
    df_stock = pd.read_csv(stock_path)
    if "week_start" in df_stock.columns:
        df_stock["week_start"] = pd.to_datetime(df_stock["week_start"]).dt.strftime("%Y-%m-%d")
    df_stock["store_id"] = df_stock["store_id"].astype(str)
    stock_availability = df_stock.to_dict("records")

    # Basic validation
    if len(customers) == 0:
        warnings.append("retail_customers.csv is empty")
    if len(weekly_sales) == 0:
        warnings.append("retail_weekly_sales.csv is empty")
    if len(stock_availability) == 0:
        warnings.append("stock_availability.csv is empty")

    snapshot_at = datetime.now(timezone.utc).isoformat()

    return {
        "customers": customers,
        "weekly_sales": weekly_sales,
        "stock_availability": stock_availability,
        "data_snapshot_loaded_at": snapshot_at,
        "validation_warnings": warnings,
    }
